In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Paths to your datasets
correct_squat_1_path = "Clean_Dataset/Correct_Squat_1.xlsx"
correct_squat_2_path = "Clean_Dataset/Correct_Squat_2.xlsx"
incorrect_squat_1_path = "Clean_Dataset/Incorrect_Squat_1.xlsx"
incorrect_squat_2_path = "Clean_Dataset/Incorrect_Squat_2.xlsx"

# Load the datasets
correct_squat_1 = pd.read_excel(correct_squat_1_path)
correct_squat_2 = pd.read_excel(correct_squat_2_path)
incorrect_squat_1 = pd.read_excel(incorrect_squat_1_path)
incorrect_squat_2 = pd.read_excel(incorrect_squat_2_path)

# Optionally add a 'Label' column if your datasets don't already include it
correct_squat_1['Label'] = 1
correct_squat_2['Label'] = 1
incorrect_squat_1['Label'] = 0
incorrect_squat_2['Label'] = 0

# Combine the datasets
all_data = pd.concat([correct_squat_1, correct_squat_2, incorrect_squat_1, incorrect_squat_2], ignore_index=True)

features = all_data.drop('Label', axis=1)
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)



##### Data Processing 

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

features = all_data.drop('Label', axis=1)
labels = all_data['Label']

scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_normalized, labels, test_size=0.2, random_state=42)

##### Define and Train the Neural Network Models

In [13]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=10, validation_split=0.1)


Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/data_adapter.py:1808: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


31/31 [==============================] - 1s 6ms/step - loss: 0.6489 - accuracy: 0.5934 - val_loss: 0.6171 - val_accuracy: 0.5688
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 0.5928 - accuracy: 0.6283 - val_loss: 0.5679 - val_accuracy: 0.6789
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 0.5478 - accuracy: 0.7372 - val_loss: 0.5217 - val_accuracy: 0.7615
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 0.5039 - accuracy: 0.7885 - val_loss: 0.4719 - val_accuracy: 0.8165
Epoch 5/10
31/31 [==============================] - 0s 3ms/step - loss: 0.4627 - accuracy: 0.8255 - val_loss: 0.4196 - val_accuracy: 0.8440
Epoch 6/10
31/31 [==============================] - 0s 3ms/step - loss: 0.4257 - accuracy: 0.8511 - val_loss: 0.3800 - val_accuracy: 0.8624
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 0.3908 - accuracy: 0.8614 - val_loss: 0.3473 - val_accuracy: 0.8532
Epoch 8/10
31/31 [=============

##### Printing performance

In [14]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

9/9 [==============================] - 0s 1ms/step - loss: 0.3084 - accuracy: 0.9077
Test Accuracy: 0.9077490568161011
